# LANGCHAIN MEDICAL CHATBOT TRIAL

#### LOADING LIBRARIES

In [17]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from pinecone import Pinecone,ServerlessSpec
from langchain_pinecone.vectorstores import PineconeVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

True

#### LOADING API KEYS

In [2]:
os.environ['HF_TOKEN']=os.getenv('HF_TOKEN')
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')
os.environ['PINECONE_API_KEY']=os.getenv('PINECONE_API_KEY')

In [3]:
path=r'C:\Users\Chetan\OneDrive\Desktop\LANGCHAIN MEDICAL CHATBOT\Medical-ChatBot\data'

#### DEFINING FUNCTIONS TO EXTRACT DATA FROM PDF AND EXTRACT ONLY MINIMAL INFORMATION 

In [4]:
def extract_data(path:str)->Document:
    document=PyPDFDirectoryLoader(path).load()
    document=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(document)
    return document

In [5]:
def minimal_extract(doc):
    for i in range(0,len(doc)-1):
        doc[i]=Document(
        page_content=doc[i].page_content,
        metadata={"source": doc[i].metadata['source']}
    )
    return doc

In [6]:
embeddings=HuggingFaceEmbeddings(model='all-MiniLM-L6-v2')
llm=ChatGroq(model='llama-3.1-8b-instant')

In [7]:
pc=Pinecone()
pc

In [8]:
index_name='medical-chatbot'

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws',region='us-east-1')
    )

index=pc.Index(index_name)

In [9]:
data=extract_data(path)
data=minimal_extract(data)

In [10]:
docsearch=PineconeVectorStore.from_documents(
    documents=data,
    embedding=embeddings,
    index_name=index_name
)

In [ ]:
## load existing index


# index=PineconeVectorStore.from_existing_index(
#     index_name=index_name,
#     embedding=embeddings
# )

## Add more data to the existing pinecone index


# doc=Document(metadata='ABC',page_content='efjbsldfkjvnsldijvcsldfkjvndsfv')
# index=PineconeVectorStore.add_documents(documents=[doc])

In [16]:
retriever=docsearch.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [18]:
system_prompt='''
    you are a medical assistant for question-answering task. use the following pieces of retrieved context to answer the question. if you dont know the answer say that you dont know. use three sentences maximum and keep the answer conscise. 
    \n\n
    context:{context}
'''
prompt=ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ('human','{input}')
    ]
)

In [19]:
qa_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,qa_chain)

In [22]:
rag_chain.invoke({'input':'What is acne?'})

{'input': 'What is acne?',
 'context': [Document(id='640306e4-b0c8-46a8-9638-30e71545d315', metadata={'source': 'C:\\Users\\Chetan\\OneDrive\\Desktop\\LANGCHAIN MEDICAL CHATBOT\\Medical-ChatBot\\data\\The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf'}, page_content='acne. Isotretinoin (Accutane) is prescribed only for\nvery severe, disfiguring acne.\nAcne is a skin condition that occurs when pores or\nhair follicles become blocked. This blockage allows a\nwaxy material called sebum to collect inside the pores\nor follicles. Normally, sebum flows out onto the skin\n240 GALE ENCYCLOPEDIA OF MEDICINE\nAntiacne drugs'),
  Document(id='d01048f3-065e-442b-8142-572f96659eaa', metadata={'source': 'C:\\Users\\Chetan\\OneDrive\\Desktop\\LANGCHAIN MEDICAL CHATBOT\\Medical-ChatBot\\data\\The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf'}, page_content='skin that houses the oil-producing glands and hair\nfollicles, where pimples form.\nSebum— 

In [23]:
from langchain_core.prompts import MessagesPlaceholder
system_prompt='''
    you are a medical assistant for question-answering task. use the following pieces of retrieved context to answer the question. if you dont know the answer say that you dont know. use three sentences maximum and keep the answer conscise. 
    \n\n
    context:{context}
'''
prompt=ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ('human','{input}')
    ]
)
history_prompt=ChatPromptTemplate.from_messages(
    [
        ('system', 'you are a medical assistant for question-answering task. use the following pieces of retrieved context to answer the question. if you dont know the answer say that you dont know. use three sentences maximum and keep the answer conscise. If the question is out of the medical domain simply say I DONT KNOW. keep it small and conscise'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human','{input}')
    ]
)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

get_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history',
    output_messages_key='answer',
)


In [25]:
get_history.invoke(
            {"input": 'I have white patches on skin and it can be itchy and flaky'},
            config={"configurable": {"session_id": 'chat'}}
        )['answer']

c:\Users\Chetan\OneDrive\Desktop\LANGCHAIN MEDICAL CHATBOT\Medical-ChatBot\.venv\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


"Based on your symptoms, it's possible that you may have a condition like vitiligo, which is a skin disorder that creates smooth, depigmented white spots on the skin. It can cause itching and flakiness. I recommend consulting a physician for a proper diagnosis and treatment plan."

In [26]:
get_history.invoke(
            {"input": 'does it mean cancer'},
            config={"configurable": {"session_id": 'chat'}}
        )['answer']

'No, malingering is feigning or exaggerating symptoms for personal gain, not the same as cancer.'

In [27]:
get_history.invoke(
            {"input": 'I also have pitting in nails'},
            config={"configurable": {"session_id": 'chat'}}
        )['answer']

'Pitting in the nails is commonly associated with psoriasis, a systemic disease and disorder listed as one of the causes of nail disorders.'

In [29]:
get_history.invoke(
            {"input": 'what problems do i have'},
            config={"configurable": {"session_id": 'chat'}}
        )['answer']

"I don't have enough information to determine the problems you have. Can you provide more context or information about your condition or symptoms?"